In [3]:
pip install minisom

  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.3-py3-none-any.whl size=11706 sha256=31fd132a88e4b87daff7941395741b1f6cd755e185ad146e0695dec70d103986
  Stored in directory: /root/.cache/pip/wheels/96/98/a5/52dee3e8ed1dbfc4d77e4da41b6d89dd7ab9ead1b921e766f8
Successfully built minisom


In [27]:
from IPython.display import display, Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from minisom import MiniSom
import time
from hyperopt import hp,tpe,fmin,Trials, STATUS_OK

In [28]:
df=pd.read_csv('PCA_data.csv',delimiter=',')
df.head()

,customer_id,customer_region_2440,customer_region_2490,customer_region_4140,customer_region_4660,customer_region_8370,customer_region_8550,customer_region_8670,customer_region_Unknown,last_promo_DISCOUNT,...,payment_method_CASH,payment_method_DIGI,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7
0,1b8f824d5e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,-0.295945,-0.445237,-0.778433,0.311961,0.054672,-0.110476,-0.141609,0.232893
1,5d272b9dcb,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,-0.296883,-0.497895,-0.761456,-0.009660,-0.092699,-0.076241,-0.158929,-0.065966
2,f6d1b2ba63,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,-0.119790,-0.444279,-0.729543,0.132792,-0.512660,0.111132,0.161004,-0.060958
3,180c632ed8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,-0.322302,-0.328141,-0.746995,0.004326,-0.071154,-0.082533,-0.019563,0.033507
4,4eb37a6705,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,-0.322450,-0.345026,-0.739681,-0.125103,-0.134561,-0.107185,-0.109648,0.229223


In [29]:
df.set_index('customer_id',inplace=True)

In [32]:
x=int(np.sqrt(5*np.sqrt(df.shape[0])))
y=x
input_len=df.shape[1]
learning_rate=0.5

In [33]:
starttime=time.time()
beat=fmin(fn=lambda sig:MiniSom(x=x,y=y,input_len=input_len,sigma=sig,learning_rate=learning_rate).quantization_error(df.values),space=hp.uniform('sig',0.001,x/2.01),algo=tpe.suggest,max_evals=200)
time_passed=time.time()-starttime
print('time passed:',time_passed)

100%|██████████| 200/200 [01:39<00:00,  2.02trial/s, best loss: 1.2488414247538988]
time passed: 99.16573810577393


In [34]:
sigma=beat['sig']
sigma

5.108662324587616

In [35]:
space={'sig':hp.uniform('sig',0.001,x/2.01),'learning_rate':hp.uniform('learning_rate',0.001,0.5)}
def son_fn(space):
  sig=space['sig']
  lr=space['learning_rate']
  val=MiniSom(x=x,y=y,input_len=input_len,sigma=sig,learning_rate=lr).quantization_error(df.values)
  return {'loss': val,'status':STATUS_OK}
trials = Trials()
best=fmin(fn=son_fn,space=space,algo=tpe.suggest,max_evals=200,trials=trials)
print('best:',format(best))

100%|██████████| 200/200 [01:39<00:00,  2.00trial/s, best loss: 1.2487430629390834]
best: {'learning_rate': 0.026142936190797994, 'sig': 2.747247410001214}


In [39]:
sigma=best['sig']
learning_rate=best['learning_rate']
sm=MiniSom(x=x,y=y,
            input_len=input_len,
            sigma=sigma,
            learning_rate=learning_rate,
            activation_distance='euclidean',
            topology='hexagonal',
            neighborhood_function='gaussian')
sm.random_weights_init(df.values)
print(np.round(sm.quantization_error(df.values),4),"Starting QE")
som.train_batch(df.values,10000)
print(np.round(sm.quantization_error(df.values),4),"Ending QE")

/usr/local/lib/python3.10/dist-packages/minisom.py:631: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(-2 * cross_term + input_data_sq + weights_flat_sq.T)


0.4353 Starting QE
0.4353 Ending QE
